# Single use key applet

This applet generates a single-use private key on a secure element (JavaCard) and can sign only one message.

The key is overwritten just before the signature is sent back to the host.

In [1]:
from applets.core import get_connection, SecureAppletBase
from applets.util import secp256k1

class SingleUseKey(SecureAppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CD01", connection)

    def generate_fresh_key(self):
        return self.secure_request(b'\x20\x00')
        
    def get_pubkey(self):
        return self.secure_request(b'\x20\x01')

    def sign(self, msg:bytes):
        if len(msg)!=32:
            raise ValueError("Message hash should be 32 bytes long")
        return self.secure_request(b'\x20\x02'+msg)

# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [4]:
app = SingleUseKey(connection)
app.select()

# establish secure communication
app.establish_secure_channel()
app.is_secure_channel_open

True

# 2. Start talking to the card securely
## 2.1. Generate a new private key
Card will return corresponding public key and will keep it until you sign something.<br>
It will overwrite the key with a new one once signature is sent to the host.

In [5]:
# generate a new random key
pub = app.generate_fresh_key()
pub.hex()

'026ea4a62f657f0ffa5deb7dfb30d039ea9e02809d9479fa9f7b364b983cf2b4f5'

In [6]:
# you can get this pubkey if you didn't sign using this key
app.get_pubkey() == pub

True

## 2.2. Sign a message with this key

Key will be deleted when the signature is sent to the host.

In [7]:
# sign message - only one message can be signed, then new key is generated
msg = b'5'*32 # some dummy message
sig = app.sign(msg)
sig.hex()

'3044022014893591426d2cf3b472dc93b2fc830b14edf7ed96ee116290eb6db29088d91502200384d89d907f7543ccef7d693f79f8cedec8ddf4ead912d847434a9ffe1c25b8'

In [8]:
# verify it is valid:
sig_raw = secp256k1.ecdsa_signature_parse_der(sig)
pub_raw = secp256k1.ec_pubkey_parse(pub)
secp256k1.ecdsa_verify(sig_raw, msg, pub_raw)

True

# 3 Close connection with the card

In [9]:
connection.disconnect()